<a href="https://colab.research.google.com/github/eduardomazevedo/moralhazard/blob/main/examples/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Quickstart demo for Azevedo Wolff (2025) "Broad Validity of the First-Order Approach in Moral Hazard"
This is a quickstart demo for the numerical examples in the paper.


Replication code: https://github.com/eduardomazevedo/azevedo-wolff-2025


Algorithm 1 implementation: https://github.com/eduardomazevedo/moralhazard

In [ ]:
# -----------------------------------------------------------------------------
# 1. SETUP & INSTALLATION
# -----------------------------------------------------------------------------
!pip install git+https://github.com/eduardomazevedo/moralhazard.git@8580160d0bcb353f00cfcd243d7ebd2f7e5f9911

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.colors import LinearSegmentedColormap
from moralhazard import MoralHazardProblem
from moralhazard.config_maker import make_utility_cfg, make_distribution_cfg
import IPython.display as display

: 

In [ ]:
# -----------------------------------------------------------------------------
# 2. DEFINE PARAMETERS
# -----------------------------------------------------------------------------
# Select which example to run:
# Options: 'log-gaussian', 'cara-gaussian', 'log-poisson', 'log-exponential', 'log-t'
EXAMPLE_TO_RUN = 'log-gaussian'

# Define parameters for each case (derived from make_figures.py)
CASES = {
    'log-gaussian': {
        'w0': 50, 'target_a': 100.0, 'a_min': 0.0, 'a_max': 180.0,
        'utility': 'log', 'theta_mult': 1.0,
        'dist_name': 'gaussian', 'dist_kwargs': {'sigma': 50.0},
        'comp_kwargs': {'distribution_type': 'continuous', 'y_min': -300.0, 'y_max': 480.0, 'n': 201},
        'res_wage_grid': np.linspace(-1.0, 15.0, 5)
    },
    'cara-gaussian': {
        'w0': 50, 'target_a': 100.0, 'a_min': 0.0, 'a_max': 180.0,
        'utility': 'cara', 'theta_mult': 10.0,
        'dist_name': 'gaussian', 'dist_kwargs': {'sigma': 50.0},
        'comp_kwargs': {'distribution_type': 'continuous', 'y_min': -300.0, 'y_max': 480.0, 'n': 201},
        'res_wage_grid': np.linspace(-1.0, 15.0, 5)
    },
    'log-poisson': {
        'w0': 50, 'target_a': 7.0, 'a_min': 0.0, 'a_max': 10.0,
        'utility': 'log', 'theta_mult': 1.0,
        'dist_name': 'poisson', 'dist_kwargs': {},
        'comp_kwargs': {'distribution_type': 'discrete', 'y_min': 0.0, 'y_max': 28.0, 'step_size': 1.0},
        'res_wage_grid': np.linspace(-1.0, 5.0, 5)
    },
    'log-exponential': {
        'w0': 50, 'target_a': 100.0, 'a_min': 10.0, 'a_max': 180.0,
        'utility': 'log', 'theta_mult': 1.0,
        'dist_name': 'exponential', 'dist_kwargs': {},
        'comp_kwargs': {'distribution_type': 'continuous', 'y_min': 0.01, 'y_max': 260.0, 'n': 201},
        'res_wage_grid': np.linspace(-1.0, 15.0, 5)
    },
    'log-t': {
        'w0': 50, 'target_a': 100.0, 'a_min': 0.0, 'a_max': 180.0,
        'utility': 'log', 'theta_mult': 1.0,
        'dist_name': 'Student_t', 'dist_kwargs': {'sigma': 20.0, 'nu': 1.15},
        'comp_kwargs': {'distribution_type': 'continuous', 'y_min': -500.0, 'y_max': 680.0, 'n': 201},
        'res_wage_grid': np.linspace(-1.0, 100.0, 5)
    }
}

# --- Apply Selection ---
params = CASES[EXAMPLE_TO_RUN]
print(f"--- Setting up: {EXAMPLE_TO_RUN} ---")

# 1. Setup Cost Function
# C(a) = theta * a^2 / 2
# Theta depends on wealth and target action
theta_base = 1.0 / params['target_a'] / (params['target_a'] + params['w0'])
theta = theta_base * params['theta_mult']

def C(a): return theta * a ** 2 / 2
def Cprime(a): return theta * a

# 2. Setup Utility
if params['utility'] == 'cara':
    # CARA specific: alpha = 1/w0
    utility_cfg = make_utility_cfg("cara", w0=params['w0'], alpha=1.0/params['w0'])
else:
    utility_cfg = make_utility_cfg("log", w0=params['w0'])

# 3. Setup Distribution and Config
dist_cfg = make_distribution_cfg(params['dist_name'], **params['dist_kwargs'])

cfg = {
    "problem_params": {**utility_cfg, **dist_cfg, "C": C, "Cprime": Cprime},
    "computational_params": params['comp_kwargs']
}

# 4. Initialize Problem
mhp = MoralHazardProblem(cfg)
y_grid = mhp._y_grid

# 5. Define Plotting Grids
a_min, a_max = params['a_min'], params['a_max']
target_action = params['target_a']
reservation_wage_grid = params['res_wage_grid']
action_grid_plot = np.linspace(a_min, a_max, 100)
n_a_iterations = 100

In [ ]:
# -----------------------------------------------------------------------------
# 3. PART A: PRINCIPAL PROBLEM (Maximize Profit)
# -----------------------------------------------------------------------------
fig_pp, (ax1_pp, ax2_pp) = plt.subplots(2, 1, figsize=(8, 8))
fig_pp.suptitle(f"Principal Problem: {EXAMPLE_TO_RUN}")

print("Solving Principal Problems...")
for rw in reservation_wage_grid:
    ru = utility_cfg["u"](rw)

    # Solve
    sol = mhp.solve_principal_problem(
        revenue_function=lambda a: a,
        reservation_utility=ru,
        a_min=a_min, a_max=a_max,
        a_ic_lb=a_min, a_ic_ub=a_max,
        n_a_iterations=n_a_iterations
    )
    print(f"Reservation wage", rw, "solved.")
    print(sol)

    # Extract Data
    contract = sol.cmp_result.optimal_contract
    wage_curve = mhp.k(contract)
    utility_curve = mhp.U(contract, action_grid_plot)
    opt_a = sol.optimal_action
    opt_u = mhp.U(contract, opt_a)
    foa_holds = sol.cmp_result.first_order_approach_holds

    # Plot
    style = '-' if foa_holds else '--'
    ax1_pp.plot(y_grid, wage_curve, linestyle=style, label=f"Res Wage {rw:.1f}")
    ax2_pp.plot(action_grid_plot, utility_curve, linestyle=style)
    ax2_pp.scatter([opt_a], [opt_u], color='red', zorder=5, s=20)

# Formatting
ax1_pp.set_title("Optimal Wage Contract w(y)")
ax1_pp.set_ylabel("Wage")
ax1_pp.set_xlim(y_grid.min(), y_grid.max()) # Auto-scale to distribution range
ax1_pp.legend()
ax1_pp.grid(True, alpha=0.3)

ax2_pp.set_title("Agent Utility vs Action")
ax2_pp.set_ylabel("Utility (Certain Equivalent)")
ax2_pp.set_xlabel("Action (a)")
ax2_pp.set_xlim(a_min, a_max)
ax2_pp.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# -----------------------------------------------------------------------------
# 4. PART B: COST MINIMIZATION (Target Action)
# -----------------------------------------------------------------------------
fig_cm, (ax1_cm, ax2_cm) = plt.subplots(2, 1, figsize=(8, 8))
fig_cm.suptitle(f"Cost Minimization: {EXAMPLE_TO_RUN} (Target a={target_action})")

print("Solving Cost Minimization Problems...")
for rw in reservation_wage_grid:
    ru = utility_cfg["u"](rw)

    # Solve
    sol = mhp.solve_cost_minimization_problem(
        intended_action=target_action,
        reservation_utility=ru,
        a_ic_lb=0.0,
        a_ic_ub=max(100.0, target_action * 1.5), # Ensure bounds cover target
        n_a_iterations=n_a_iterations
    )
    print(f"Reservation wage", rw, "solved.")
    print(sol)

    # Extract Data
    contract = sol.optimal_contract
    wage_curve = mhp.k(contract)
    utility_curve = mhp.U(contract, action_grid_plot)
    foa_holds = sol.first_order_approach_holds
    u_at_target = mhp.U(contract, target_action)

    # Plot
    style = '-' if foa_holds else '--'
    ax1_cm.plot(y_grid, wage_curve, linestyle=style, label=f"Res Wage {rw:.1f}")
    ax2_cm.plot(action_grid_plot, utility_curve, linestyle=style)
    ax2_cm.scatter([target_action], [u_at_target], color='red', zorder=5, s=20)

# Formatting
ax1_cm.set_title("Optimal Wage Contract w(y)")
ax1_cm.set_ylabel("Wage")
ax1_cm.set_xlim(y_grid.min(), y_grid.max())
ax1_cm.legend()
ax1_cm.grid(True, alpha=0.3)

ax2_cm.set_title("Agent Utility vs Action")
ax2_cm.set_ylabel("Utility (Certain Equivalent)")
ax2_cm.set_xlabel("Action (a)")
ax2_cm.set_xlim(a_min, a_max)
ax2_cm.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# -----------------------------------------------------------------------------
# 5. PART C: COMPARISON WITH CVXPY (Lowest Reservation Utility)
# -----------------------------------------------------------------------------
# Take the case with the lowest reservation utility
lowest_res_wage = reservation_wage_grid.min()
lowest_ru = utility_cfg["u"](lowest_res_wage)

print(f"Comparing Algorithm 1 (Dual) vs CVXPY at reservation wage = {lowest_res_wage:.1f}")

# Create a_hat grid for CVXPY (same as action_grid_plot but 101 points)
a_hat_cvxpy = np.linspace(a_min, a_max, 101)

# Solve with Algorithm 1 (Dual)
print("Solving with Algorithm 1...")
sol_dual = mhp.solve_cost_minimization_problem(
    intended_action=target_action,
    reservation_utility=lowest_ru,
    a_ic_lb=a_min,
    a_ic_ub=a_max,
    n_a_iterations=n_a_iterations
)
print(f"  Expected wage: {sol_dual.expected_wage:.4f}")

# Solve with CVXPY
print("Solving with CVXPY...")
sol_cvxpy = mhp.solve_cost_minimization_problem_cvxpy(
    intended_action=target_action,
    reservation_utility=lowest_ru,
    a_hat=a_hat_cvxpy,
)
print(f"  Expected wage: {sol_cvxpy['expected_wage']:.4f}")

# Extract data for both
v_dual = sol_dual.optimal_contract
w_dual = mhp.k(v_dual)
U_dual = mhp.U(v_dual, action_grid_plot)

v_cvxpy = sol_cvxpy['v']
w_cvxpy = mhp.k(v_cvxpy)
U_cvxpy = mhp.U(v_cvxpy, action_grid_plot)

# Create figure with 3 subplots
fig_compare, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 10))
fig_compare.suptitle(f"Algorithm Comparison: {EXAMPLE_TO_RUN}\n(Reservation Wage = {lowest_res_wage:.1f}, Target a = {target_action})")

# Plot 1: v vs y (Optimal Contract)
ax1.plot(y_grid, v_dual, '-', linewidth=2, label='Algorithm 1')
ax1.plot(y_grid, v_cvxpy, '--', linewidth=2, label='CVXPY')
ax1.set_title("Optimal Contract v(y)")
ax1.set_ylabel("v (Utility Units)")
ax1.set_xlabel("Output (y)")
ax1.set_xlim(y_grid.min(), y_grid.max())
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: w = k(v) vs y (Wage Schedule)
ax2.plot(y_grid, w_dual, '-', linewidth=2, label='Algorithm 1')
ax2.plot(y_grid, w_cvxpy, '--', linewidth=2, label='CVXPY')
ax2.set_title("Wage Schedule w(y) = k(v(y))")
ax2.set_ylabel("Wage")
ax2.set_xlabel("Output (y)")
ax2.set_xlim(y_grid.min(), y_grid.max())
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: U vs a (Agent Utility)
ax3.plot(action_grid_plot, U_dual, '-', linewidth=2, label='Algorithm 1')
ax3.plot(action_grid_plot, U_cvxpy, '--', linewidth=2, label='CVXPY')
ax3.axvline(x=target_action, color='red', linestyle=':', alpha=0.7, label=f'Target a={target_action}')
ax3.set_title("Agent Utility vs Action")
ax3.set_ylabel("Utility (Certain Equivalent)")
ax3.set_xlabel("Action (a)")
ax3.set_xlim(a_min, a_max)
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary
print("\n--- Summary ---")
print(f"Algorithm 1 expected wage: {sol_dual.expected_wage:.4f}")
print(f"CVXPY expected wage:       {sol_cvxpy['expected_wage']:.4f}")
print(f"Difference:                {abs(sol_dual.expected_wage - sol_cvxpy['expected_wage']):.6f}")